In [2]:
import utils

import pandas as pd
import re

from sklearn import preprocessing

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 12, 8
%matplotlib inline

In [51]:
!head PM_train.txt

1 1 -0.0007 -0.0004 100.0 518.67 641.82 1589.70 1400.60 14.62 21.61 554.36 2388.06 9046.19 1.30 47.47 521.66 2388.02 8138.62 8.4195 0.03 392 2388 100.00 39.06 23.4190  
1 2 0.0019 -0.0003 100.0 518.67 642.15 1591.82 1403.14 14.62 21.61 553.75 2388.04 9044.07 1.30 47.49 522.28 2388.07 8131.49 8.4318 0.03 392 2388 100.00 39.00 23.4236  
1 3 -0.0043 0.0003 100.0 518.67 642.35 1587.99 1404.20 14.62 21.61 554.26 2388.08 9052.94 1.30 47.27 522.42 2388.03 8133.23 8.4178 0.03 390 2388 100.00 38.95 23.3442  
1 4 0.0007 0.0000 100.0 518.67 642.35 1582.79 1401.87 14.62 21.61 554.45 2388.11 9049.48 1.30 47.13 522.86 2388.08 8133.83 8.3682 0.03 392 2388 100.00 38.88 23.3739  
1 5 -0.0019 -0.0002 100.0 518.67 642.37 1582.85 1406.22 14.62 21.61 554.00 2388.06 9055.15 1.30 47.28 522.19 2388.04 8133.80 8.4294 0.03 393 2388 100.00 38.90 23.4044  
1 6 -0.0043 -0.0001 100.0 518.67 642.10 1584.47 1398.37 14.62 21.61 554.67 2388.02 9049.68 1.30 47.16 521.68 2388.03 8132.85 8.4108 0.03 391 2388 100.00 38.98 

In [40]:
import pandas as pd
import re

from sklearn import preprocessing

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 12, 8
%matplotlib inline

In [41]:
re.search(r's\d+', 'sd') is not None

False

In [190]:
colnames = ["id","cycle","setting1","setting2","setting3","s1","s2","s3","s4","s5","s6","s7",
           "s8","s9","s10","s11","s12","s13","s14","s15","s16","s17","s18","s19","s20","s21"]
# df = pd.read_table('PM_train.txt', sep='\s+', names=colnames, index_col='id')
df = pd.read_table('PM_train.txt', sep='\s+', names=colnames)

In [ ]:
sensor_colnames = [x for x in df.columns if x == 'id' or re.search('s\d', x) is not None]
sensor_df = df[sensor_colnames]
sensor_df.groupby('id').transform(lambda grp: grp.rolling(5, min_periods=1).mean())

In [43]:
df.head()

,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
id,,,,,,,,,,,,,,,,,,,,,
1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


### Generate column RUL (Remaining Useful Life)

In [200]:
f = lambda x: 2 if x <= w0 else (1 if x <= w1 else 0)

In [205]:
f(31)

0

In [44]:
# Window for different target classes
w1 = 30
w0 = 15

def assign_rul(group):
    return group.max() - group

def assign_label1(val):
    if val <= w1:
        return 1
    else:
        return 0
    
def assign_label2(val):
    if val <= w0:
        return 2
    elif val <= w1:
        return 1
    else:
        return 0

df['RUL'] = df.groupby(level='id')['cycle'].transform(assign_rul)
df['RUL'] = df.groupby(level='id').transform(assign_rul)
df['label1'] = df['RUL'].apply(assign_label1)
df['label2'] = df['RUL'].apply(assign_label2)
df.head()

,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,...,s15,s16,s17,s18,s19,s20,s21,RUL,label1,label2
id,,,,,,,,,,,,,,,,,,,,,
1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,191.0,0,0
1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,190.0,0,0
1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,189.0,0,0
1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,188.0,0,0
1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,187.0,0,0


In [45]:
filter_sensor_cols = [x for x in df.columns if re.search('s\d', x) is not None]
sensor_data = df[filter_sensor_cols]

In [46]:
df_rollmean = sensor_data.groupby(level='id', as_index=False).apply(lambda grp: grp.rolling(5, min_periods=1).mean())
df_rollmean.columns = ['{}_rollmean'.format(x) for x in df_rollmean.columns]
df_rollstd = sensor_data.groupby(level='id', as_index=False).apply(lambda grp:grp.rolling(5, min_periods=1).std()).fillna(0)
df_rollstd.columns = ['{}_rollstd'.format(x) for x in df_rollstd.columns]

In [47]:
df = pd.concat([df, df_rollmean, df_rollstd], axis=1)

In [35]:
# Normalize with min_max
def min_max(col):
    minval = min(col)
    maxval = max(col)
    rangeval = maxval - minval
    return (col - minval) / rangeval

cols_for_normalization = [x for x in df.columns if x not in ['id', 'label1', 'label2']]
df = df[cols_for_normalization].apply(min_max).fillna(0).dropna(how='all')

In [36]:
df[:5]

,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,...,s12_rollstd,s13_rollstd,s14_rollstd,s15_rollstd,s16_rollstd,s17_rollstd,s18_rollstd,s19_rollstd,s20_rollstd,s21_rollstd
id,,,,,,,,,,,,,,,,,,,,,
1,0.00000,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,0.00277,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,...,0.488189,0.160656,0.508246,0.132258,0.0,0.000000,0.0,0.0,0.130435,0.017365
1,0.00554,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,...,0.450405,0.120224,0.374753,0.116172,0.0,0.408248,0.0,0.0,0.169323,0.237961
1,0.00831,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,...,0.552267,0.133773,0.307559,0.427568,0.0,0.353553,0.0,0.0,0.234642,0.202745
1,0.01108,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,...,0.481694,0.117620,0.267278,0.394651,0.0,0.387298,0.0,0.0,0.226338,0.178837


In [49]:
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(df[cols_for_normalization])
df[cols_for_normalization] = min_max_scaler.transform(df[cols_for_normalization])

In [53]:
# Select features based on Pearson correlation
from scipy.stats import pearsonr

In [121]:
features = df[[x for x in cols_for_normalization if x != 'RUL']]

In [127]:
features['s2'].corr(df['RUL'])

-0.60648397437833812

In [161]:
feature_cols = [x[0] for x in sorted(corr, key=lambda x: abs(x[1]),reverse=True)[:35]]

In [164]:
train_X, train_y = df[feature_cols].values, df['RUL'].values

In [166]:
train_X.shape, train_y.shape

((20631, 35), (20631,))

In [167]:
from sklearn.ensemble import RandomForestRegressor

In [169]:
estimator = RandomForestRegressor(n_estimators=8, max_depth=32)
estimator.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=32,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=8, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [153]:
from __future__ import print_function
for x in sorted(corr, key=lambda tupl: abs(tupl[1]), reverse=True):
    print(x[0])

cycle
s4_rollmean
s11_rollmean
s21_rollmean
s15_rollmean
s20_rollmean
s17_rollmean
s12_rollmean
s7_rollmean
s2_rollmean
s3_rollmean
s11
s4
s12
s7
s15
s21
s20
s2
s17
s8_rollmean
s13_rollmean
s3
s8
s13
s9_rollmean
s9
s14_rollmean
s14
s6_rollstd
s6_rollmean
s6
s14_rollstd
s9_rollstd
s13_rollstd
s11_rollstd
s20_rollstd
s4_rollstd
s12_rollstd
s17_rollstd
s8_rollstd
s15_rollstd
s7_rollstd
s2_rollstd
s3_rollstd
s21_rollstd
setting1
setting2


In [147]:
import numpy as np
corr = []
for col in features.columns:
    corrval = (col, features[col].corr(df['RUL']))
    if not np.isnan(corrval[1]):
        corr.append(corrval)

In [122]:
df['corr'] = features.corrwith(df['RUL'])

In [123]:
df[:5]

,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,...,s13_rollstd,s14_rollstd,s15_rollstd,s16_rollstd,s17_rollstd,s18_rollstd,s19_rollstd,s20_rollstd,s21_rollstd,corr
id,,,,,,,,,,,,,,,,,,,,,
1,0.00000,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,NaN
1,0.00277,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,...,0.160656,0.508246,0.132258,0.0,0.000000,0.0,0.0,0.130435,0.017365,NaN
1,0.00554,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,...,0.120224,0.374753,0.116172,0.0,0.408248,0.0,0.0,0.169323,0.237961,NaN
1,0.00831,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,...,0.133773,0.307559,0.427568,0.0,0.353553,0.0,0.0,0.234642,0.202745,NaN
1,0.01108,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,...,0.117620,0.267278,0.394651,0.0,0.387298,0.0,0.0,0.226338,0.178837,NaN


In [119]:
features['cycle'].corr(features['s2'])

0.5498980406467342

In [106]:
pearsonr([1,2,5], [3,4,3])

(-0.27735009811261446, 0.82108762497793275)

In [159]:
text = "RUL	cycle	a4	a11	a21	a15	a20	a17	a12	a7	a2	a3	s11	s4	s12	s7	s15	s21	s20	s2	s17	a8	a13	s3	s8	s13	a9	s9	a14	s14	sd6	a6	s6	sd14	sd9	sd13"
cols = []
for c in text.split():
    if c.startswith('a'):
        c = 's' + c[1:] + '_rollmean'
    elif c.startswith('sd'):
        c = 's' + c[2:] + '_rollstd'
    cols.append(c)

for c in cols:
    print(c)

RUL
cycle
s4_rollmean
s11_rollmean
s21_rollmean
s15_rollmean
s20_rollmean
s17_rollmean
s12_rollmean
s7_rollmean
s2_rollmean
s3_rollmean
s11
s4
s12
s7
s15
s21
s20
s2
s17
s8_rollmean
s13_rollmean
s3
s8
s13
s9_rollmean
s9
s14_rollmean
s14
s6_rollstd
s6_rollmean
s6
s14_rollstd
s9_rollstd
s13_rollstd


In [198]:
truth = pd.read_table('PM_truth.txt', sep='\s+', names=['RUL'])
truth[:3]

,RUL
0,112
1,98
2,69


In [206]:
def add_labels(df, w0=15, w1=30):
    df['label1'] = df['RUL'].apply(lambda x: 1 if x <= w1 else 0)
    df['label2'] = df['RUL'].apply(lambda x: 2 if x <= w0 else (1 if x <= w1 else 0))
    return df

In [207]:
add_labels(truth)

,RUL,label1,label2
0,112,0,0
1,98,0,0
2,69,0,0
3,82,0,0
4,91,0,0
5,93,0,0
6,91,0,0
7,95,0,0
8,111,0,0
9,96,0,0


In [208]:
def add_rul(df):
    # Remaining useful life
    df['RUL'] = df.groupby('id')['cycle'].transform(lambda grp: grp.max() - grp)
    return df


def add_labels(df, w0=15, w1=30):
    df['label1'] = df['RUL'].apply(lambda x: 1 if x <= w1 else 0)
    df['label2'] = df['RUL'].apply(lambda x: 2 if x <= w0 else (1 if x <= w1 else 0))
    return df


def add_rolling_values(agg_func, result_prefix, df, window_size=5):
    # Sensor data all in s1,s2,s3... pattern
    sensor_colnames = [x for x in df.columns if x == 'id' or re.search('s\d', x) is not None]
    sensor_df = df[sensor_colnames]
    new_df = sensor_df.groupby('id').transform(agg_func).fillna(0)
    # Column names for rolling mean will be a1,a2,a3...
    new_df.columns = [result_prefix + x[1:] for x in new_df.columns]
    df = pd.concat([df, new_df], axis=1)
    return df


def select_top_features(df, measure='pearson_correlation', n=35):
    if measure != 'pearson_correlation':
        raise ValueError('Only pearson correlation is supported for now.')
    excluded_colnames = ['id', 'label1', 'label2', 'RUL']
    feature_colnames = [x for x in df.columns if x not in excluded_colnames]
    feature_corr = []
    for col in feature_colnames:
        corr = df[col].corr(df['RUL'])
        if not np.isnan(corr):
            feature_corr.append((col, corr))
    top_corr = sorted(feature_corr, key=lambda x: abs(x[1]), reverse=True)[:n]
    return [colname for colname, val in top_corr]

In [239]:
window_size = 5
df_test = pd.read_table('PM_test.txt', sep='\s+', names=colnames)
df_truth = pd.read_table('PM_truth.txt', sep='\s+', names=['RUL'])

In [240]:
df_test = add_rolling_values(lambda grp: grp.rolling(window_size, min_periods=1).mean(), 'a', df_test)
df_test = add_rolling_values(lambda grp: grp.rolling(window_size, min_periods=1).std(), 'sd', df_test)

In [242]:
df_test = df_test[df_test.groupby('id')['cycle'].transform(max) == df_test['cycle']]

In [ ]:
df_test.index = range(len(df_test))
df_test

In [244]:
# Append RUL, label1, label2 to test data
df_truth = add_labels(df_truth)

In [257]:
pd.concat([df_test,df_truth], axis=1)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,sd15,sd16,sd17,sd18,sd19,sd20,sd21,RUL,label1,label2
0,1,31,-0.0006,0.0004,100.0,518.67,642.58,1581.22,1398.91,14.62,...,0.021174,0.0,0.707107,0.0,0.0,0.124016,0.025037,112,0,0
1,2,49,0.0018,-0.0001,100.0,518.67,642.55,1586.59,1410.83,14.62,...,0.011429,0.0,1.095445,0.0,0.0,0.069785,0.027820,98,0,0
2,3,126,-0.0016,0.0004,100.0,518.67,642.88,1589.75,1418.89,14.62,...,0.030924,0.0,0.836660,0.0,0.0,0.143631,0.018966,69,0,0
3,4,106,0.0012,0.0004,100.0,518.67,642.78,1594.53,1406.88,14.62,...,0.021249,0.0,1.341641,0.0,0.0,0.126372,0.021106,82,0,0
4,5,98,-0.0013,-0.0004,100.0,518.67,642.27,1589.94,1419.36,14.62,...,0.009627,0.0,0.547723,0.0,0.0,0.091924,0.101100,91,0,0
5,6,105,0.0076,-0.0003,100.0,518.67,643.05,1586.94,1404.49,14.62,...,0.007488,0.0,0.447214,0.0,0.0,0.153395,0.071794,93,0,0
6,7,160,0.0016,-0.0001,100.0,518.67,642.10,1589.59,1413.57,14.62,...,0.016718,0.0,0.000000,0.0,0.0,0.098387,0.063694,91,0,0
7,8,166,0.0016,-0.0005,100.0,518.67,642.59,1591.91,1413.89,14.62,...,0.018301,0.0,0.447214,0.0,0.0,0.054314,0.036437,95,0,0
8,9,55,-0.0003,0.0004,100.0,518.67,642.27,1593.61,1410.27,14.62,...,0.013525,0.0,0.836660,0.0,0.0,0.015166,0.047928,111,0,0
9,10,192,-0.0018,0.0004,100.0,518.67,643.00,1589.50,1398.99,14.62,...,0.030489,0.0,0.836660,0.0,0.0,0.087920,0.055861,96,0,0


In [248]:
pd.concat([df_test, df_truth], axis=1)

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,sd15,sd16,sd17,sd18,sd19,sd20,sd21,RUL,label1,label2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0,0.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.0,0.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,0.0,0.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,0.0,0.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.0,0.0,0.0


In [245]:
df_truth

,RUL,label1,label2
0,112,0,0
1,98,0,0
2,69,0,0
3,82,0,0
4,91,0,0
5,93,0,0
6,91,0,0
7,95,0,0
8,111,0,0
9,96,0,0
